In [ ]:
#conda activate scan_snpeff

In [30]:
import pandas as pd
from pathlib import Path
import os
import numpy as np

In [10]:
proj_dir="/master/nplatt/sch_hae_scan"

In [13]:
Path("{}/results/snpeff".format(proj_dir)).mkdir(parents=True, exist_ok=True)
os.chdir("{}/results/snpeff".format(proj_dir))

In [ ]:
%%bash 

conda run -n scan_snpeff --cwd . --live-stream \
    java -jar snpEff.jar build -noCheckCds -noCheckProtein -gff3 -v GCF_000699445.3

In [ ]:
%%bash 

conda run -n scan_snpeff --cwd . --live-stream \
    java -jar -Xmx64g snpEff.jar \
        ann \
        -csvStats snpeff_stats.csv \
        -i vcf \
        -o vcf \
        -no-downstream \
        -no-intergenic \
        -no-intron \
        -no-upstream \
        -fastaProt snpeff_prots.fas \
        GCF_000699445.3 \
        ../phasing/beagle.vcf \
        >snpeff.vcf

In [15]:
pca_df=pd.read_csv(f"{proj_dir}/results/pca/pca_df.csv", sep=",")


,sample_name,1,2,3,4,5,6,7,8,9,...,locale,host,phenotype,project,data_type,lat,lon,kmeans_group,pca_species,kmeans_label
0,Sb_NG_au_1.2,-0.186571,0.028344,0.002659,0.004098,-0.070428,-0.050869,0.009811,-0.013460,-0.016134,...,na,cow,bovis,nigeria,genome,7.067448,6.271139,2,s_bovis,sb
1,Sb_NG_au_2.10,-0.186390,0.027938,0.002539,0.005840,-0.070521,-0.050565,0.007871,-0.013283,-0.014293,...,na,cow,bovis,nigeria,genome,7.067448,6.271139,2,s_bovis,sb
2,Sb_NG_au_2.13,-0.186938,0.028149,0.002587,0.002080,-0.073837,-0.052819,0.006287,-0.017596,-0.015425,...,na,cow,bovis,nigeria,genome,7.067448,6.271139,2,s_bovis,sb
3,Sb_NG_au_2.5,-0.186747,0.027939,0.003308,-0.001183,-0.072298,-0.050190,0.007316,-0.011898,-0.008823,...,na,cow,bovis,nigeria,genome,7.067448,6.271139,2,s_bovis,sb
4,Sb_NG_au_2.6,-0.186556,0.028644,0.001854,0.006496,-0.069384,-0.049374,0.007875,-0.015933,-0.022326,...,na,cow,bovis,nigeria,genome,7.067448,6.271139,2,s_bovis,sb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,ssp_niger_libore_166,0.021444,-0.079803,-0.018635,-0.066947,0.003088,0.018973,0.086455,-0.034980,-0.046315,...,na,snail,unk,scan_africa,genome,13.388050,2.199130,0,s_haematobium,sh_nw
162,ssp_niger_libore_167,0.022669,-0.077370,-0.009122,-0.064440,-0.003224,0.026673,0.059154,0.003149,-0.002788,...,na,snail,unk,scan_africa,genome,13.390080,2.197110,0,s_haematobium,sh_nw
163,ssp_niger_libore_168,0.020863,-0.077872,-0.016320,-0.064476,-0.005623,-0.001633,0.037675,-0.009439,-0.063435,...,na,snail,unk,scan_africa,genome,13.390080,2.197110,0,s_haematobium,sh_nw
164,ssp_niger_libore_169,0.022264,-0.076285,-0.019965,-0.053381,-0.012734,0.007188,0.066627,0.009003,-0.007182,...,na,snail,unk,scan_africa,genome,13.404170,2.188020,0,s_haematobium,sh_nw


In [ ]:
#get the genetic data
vcf_reader = vcf.Reader(open('snpeff.vcf', 'r'))
#get the sample order
samples=vcf_reader.samples

In [33]:
pops=["sb", "sh_nw", "sh_se"]

for pop in pops:
    a = pca_df.loc[ pca_df["kmeans_label"] == pop, "sample_name"].values
    np.savetxt(f"{pop}.list", a, fmt="%s")
    
    cmd = f"vcftools --vcf snpeff_beagle.vcf --bed ../rfmix/merged_high_sb_regions.bed --freq --keep {pop}.list --stdout >{pop}.high_sb.freq"
    
    !conda run -n scan_snpeff --cwd . --live-stream {cmd}

In [ ]:
vcftools \
    --vcf snpeff_beagle.vcf \
    --bed merged_high_sb_regions.bed \
    --recode \
    --recode-INFO-all \
    --stdout \
    >peak.vcf

In [ ]:
vcftools \
    --vcf snpeff_beagle.vcf \
    --bed merged_high_sb_regions.bed \
    --freq \
    --keep <SB> \
    --stdout \
    >sb_high_sb.freq

In [ ]:
vcftools \
    --vcf snpeff_beagle.vcf \
    --bed merged_high_sb_regions.bed \
    --freq \
    --keep <SB> \
    --stdout \
    >peak.vcf

In [ ]:
vcftools \
    --vcf snpeff_beagle.vcf \
    --bed merged_high_sb_regions.bed \
    --freq \
    --keep <SB> \
    --stdout \
    >peak.vcf

In [ ]:
MS3_00007802

`hypothetical protein MS3_00007802 [Schistosoma haematobium]` on NCBI has the same AA seq as `MS3_0012553 Leishmanolysin-like peptidase` on parasite. (See below). But these appear to occur at different positions on the genome. Likely due to a difference in assemblies?

- MS3_00007802 = scaffold020018:8,567,422-8,600,894 (which is chr4)
- MS3_0012553 = CM039146.1:28478765-28489490 (which is chr4)

```>aa
MWKIKDIILFPIFGVHFIISGLRYDRFSTLCHHGVINHSEVVTHVVSSDVQKLIRRNSQT
ELRIHTHYDRSIKSLKNFQDIKRDVIEVAIEFWEDALLLRQSHTVPRRLTLDRVCVDGAT
KLMTFDNQLLTFCVNGCVEWTKCGPINIPVSHLNCRFIYNGTSMISGRKGEGIYRANFIL
YISSLRTHRCNTAKVLGYAAHCQLEANTDRPMAGYINFCPDTLSNEYNDKMRSLFVATHE
ILHSLGFSTSLFAFYRDKQNRPLTPRDPITFKPALGWYSGKNGQVYQWSDNVVRSVNRTW
LSAFGTFQKLAHIVVLPTVVRIARSFYNCPNLDGVELEDEDDAGVYLTHWEKRLLENDLM
TATYTNSFRISPITLAMMEDTGGHPIAPFCQQLTSSFHNKNNGLQVSCTPGEESYGFCNL
IEYSKPLPDEYVYFVNTNFSTVTLNSEINLGTGNLTNQYPLVKLGGKIALANYCPYHQEI
EWEDDVGKSIANTHCHASTNLKDPHHNFKLERFGMDSVCVEHSPNWYLINGDSLFVPPVE
GAGCYTFRCSMTEGGLILELAGGMSIPCEVPGELSEINACLTKQSLTVKGKLVCPDCRLL
CPLSECPKIYSISQNNVTYLRSNYSNDRSSSFLYETNMFIPKSRTIQYSQLIVNETESIV
ENPLYIVENSKNSKGIVIQQAILPGACLVNIAYSSTSLTFSIFIYFVVFVQLFCL```

In [ ]:
#how many of these genes in the high_sb_regions have non-synonymous changes

In [ ]:
vcftools \
    --vcf snpeff.vcf \
    --chr NC_067199.1 \
    --from-bp 28466881 \
    --to-bp 28497752 \
    --recode \
    --recode-INFO-all \
    --stdout \
    >invad.vcf

In [ ]:
vcftools \
    --vcf invad.vcf \
    --keep nw.list \
    --freq \
    --stdout \
    >nw.freq